## Abstractive Text Summarization
This notebook peforms an end to end solution to perform abstractive text summarization on CNN dailymail dataset.

## Model Building and Testing

### Importing Libraries
We first install required packages in the colab environment to train the model.
We then import all the packages and modules for later use.

In [ ]:
!pip install transformers datasets evaluate rouge_score nltk tensorflow -q

import datasets
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, AdamWeightDecay
from transformers.keras_callbacks import KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from datasets import load_dataset, load_metric
import requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.2 MB/s eta 0:00:00


### Downloading Dataset

*   We first load the cnn_dailmail dataset
*   Contents of the news_dataset:
  *  Article - The text part of the article
  *  Highlights - The summary of the article
  *  Id - unique id for the article (hash value)



In [ ]:
news_datasets = load_dataset("cnn_dailymail",'3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
news_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

### Loading metric
* ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a commonly used metric to evaluate the quality of generated summaries by comparing them to reference summaries or human-generated summaries.
* ROUGE measures the overlap between the generated text and the reference summaries in terms of n-grams, word sequences, and other linguistic units.
* It typically includes several variations of the metric, such as
  * ROUGE-N - for measuring n-gram overlap
  * ROUGE-L - for measuring longest common subsequence
  * ROUGE-W  - for measuring weighted word overlap

In [ ]:
metric = load_metric("rouge")

<ipython-input-4-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


### Tokenizer
* We first define the pre-defined model we want use, here it is T5-small (t2t transfer transformer.
* We then initialize a tokenizer using the pre-trained model specified by the model_checkpoint. The AutoTokenizer.from_pretrained method loads the tokenizer corresponding to t5 model.
* The tokenizer is used to preprocess and tokenize text data, making it suitable for input to our model. It converts text to tokens, padding sequences, and converts tokens back to text.
* So in a way tokenizer is also a part of pre-processing before the model receives it. The articles are passed throught this and then to the model.

In [ ]:
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

### Pre-processing
*  In case of T5-small model we have to prefix the inputs with "summarize: " so that the model can understnad what task to peform. Since it is already a part of the model, we add the prefix to the text and send it along.
*  We then apply the tokenizer which was initiated previously on the train, validation and test datasets.

In [ ]:
prefix = "summarize: "

In [ ]:
max_input_length = 1024
max_target_length = 128

In [ ]:
def preprocess_function(dataset):
    inputs = [prefix + doc for doc in dataset["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(text_target=dataset["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = news_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

* After tokeinizing the dataset it looks in this format -
  * Article - the text of article (already exists)
  * Highlights - the summary of the article (already exists)
  * Id - unique id of the articel (already exists)
  * input_ids - the tokenized form of the article (after preproc)
  * attention_mask - a binary mask defined by the t5-small (after preproc)
  * labels - the tokenized form of highlights (after preproc)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

### Fine Tuning the Model
* Since, we preprocessd the data we can download the pretrained model and fine tune it on the data.
* We use seq2seq because both the input and output are text sequences and we initialize the Seq2Seq language model that is pre-trained on a specific task using the architecture and parameters specified by the model_checkpoint.
* In this case that would be t5-small which was already defined.

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
# Defining hyperparameters
batch_size = 8 # how many samples in each training step
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 5

### Padding and data collating
* Data collators are used to prepare and batch input data for training and inference.
* Here we use built-in data collater for seq2seq using the tokenizer and model that we specified. We specify that it should return numpy arrays as output.
* We also use generation data collator which can pad upto multiples of 128 so that all seqeunces can be similar shape.

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np")

generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np", pad_to_multiple_of=128)

### Converting to TF dataset format
* We now prepare the dataset for seq2seq model.
* We use model.prepare_tf_dataset to ensure that the tokenized datasets are transformed into a format that's compatible with the defined Seq2Seq model.
* We can specify batched data and proper data collation for each set and these datasets can then be used for training and evaluation.

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=generation_data_collator
)

### Compiling the model
* We define the model optimizer as adam with weight decay. It is a regularization technique that adds a penalty to the models weight to prevent it from over-fitting. The parameters are predefined.

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

#### Metric function
* The metric_fn function is used to compute evaluation metrics for text summarization, particularly using the ROUGE metric.
* We first read the eval_predictions containing the generated pred, labels
* we then decode them into human format.
* We then split them by the lines using nltk.sent_tokenize as roque uses sentence wise comparision.
* Then rogue metric is then computed and returned.
* The mean length of the predictions is calculated so we can use later for evaluation.

In [ ]:
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(predictions=decoded_predictions, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return result

### Checkpoints and Callbacks
* We initiated tensor board callback that logs metrics and can be seen using tensorboard at a later time.
* By using KerasMetricCallback we can track custom metrics generated from metric_fn. We can specify xla_generation for faster optimization on generation dataset.
* Finally we define the model_checkpoint to save the model while monitoring the validation loss as a metric.

In [ ]:
tensorboard_callback = TensorBoard(log_dir="./abstractive_model/logs")
metric_callback = KerasMetricCallback(metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True)
model_checkpoint_callback=ModelCheckpoint(filepath='abstractive.h5', save_weights_only=True, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
callbacks = [metric_callback, tensorboard_callback, model_checkpoint_callback]

### Training the model

In [ ]:
N_STEPS = int((287113*0.001))//batch_size

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset,steps_per_epoch=N_STEPS, epochs=num_train_epochs, callbacks=callbacks)

Epoch 1/5
35/35 [==============================] - ETA: 0s - loss: 2.3421 

KeyboardInterrupt: ignored

### Saving the model in tf format
Since the model was not completly trained for the given set of epochs and interrupted, I saved the model using model.save in tf format. This format can be used for TF serving as well and is lightweight.

In [ ]:
!mkdir -p saved_model
model.save('saved_model/T5_abs_summ')

### Testing the model
* To test the model we take an article from test data. We prefix it with "summarize: ".
* Like we did in the pre-processing, we tokenize the article before sending it to the model for generation.
* The tokenized article is now sent as arguments to model.generate, to generated text.
* We now decode the output of the generated text into human readable text format.

In [ ]:
article=news_datasets['test'][0]['article']
print("Article: ")
print(article)
article = "summarize: " + article
tokenized = tokenizer([article], max_length=max_input_length, truncation=True,return_tensors='np')
out = model.generate(**tokenized, max_length=max_target_length)

Article: 
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ce

In [ ]:
print("Summary:")
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

Summary:
<pad> the Palestinian Authority officially became the 123rd member of the International Criminal Court. the ICC opened a preliminary examination into the situation in Palestinian territories. Israel and the united states opposed the Palestinians' efforts to join the body.</s>


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


#### Baseline News Article for comparing similarity in abstractive and extractive methods. The same is used for testing TF serving.

In [ ]:
article = 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we\'re neglected or forgotten," she said.\n"That may not be true but it is perhaps my perspective over the last few days.\n"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"\nMeanwhile, a flood alert remains in place across the Borders because of the constant rain.\nPeebles was badly hit by problems, sparking calls to introduce more defences in the area.\nScottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.\nThe Labour Party\'s deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.\nHe said it was important to get the flood protection plan right but backed calls to speed up the process.\n"I was quite taken aback by the amount of damage that has been done," he said.\n"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."\nHe said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.\nHave you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.'
article = "summarize: " + article

In [ ]:
tokenized = tokenizer([article], max_length=max_input_length, truncation=True,return_tensors='np')
out = model.generate(**tokenized, max_length=max_target_length)

In [ ]:
print("Summary:")
with tokenizer.as_target_tokenizer():
   print(tokenizer.decode(out[0]))

Summary:
<pad> repairs are ongoing in Hawick and many roads in Peeblesshire remain badly affected. many businesses and householders were affected by flooding in Newton Stewart. the waters breached a retaining wall, flooding many commercial properties.</s>


## Serving the Summarizer

### Saving the model and exploring saved_model

In [ ]:
import tempfile
import os
import subprocess
MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1


Saved model:
total 8032
drwxr-xr-x 2 root root    4096 Nov  7 06:31 assets
-rw-r--r-- 1 root root      56 Nov  7 06:31 fingerprint.pb
-rw-r--r-- 1 root root  204017 Nov  7 06:31 keras_metadata.pb
-rw-r--r-- 1 root root 8005297 Nov  7 06:31 saved_model.pb
drwxr-xr-x 2 root root    4096 Nov  7 06:31 variables


In [ ]:
!zip -r /content/abs.zip /tmp/1
from google.colab import files
files.download("/content/abs.zip")

  adding: tmp/1/ (stored 0%)
  adding: tmp/1/assets/ (stored 0%)
  adding: tmp/1/keras_metadata.pb (deflated 96%)
  adding: tmp/1/fingerprint.pb (stored 0%)
  adding: tmp/1/saved_model.pb (deflated 93%)
  adding: tmp/1/variables/ (stored 0%)
  adding: tmp/1/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: tmp/1/variables/variables.index (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# contents of the saved_model
!saved_model_cli show --dir {export_path} --all

2023-11-07 06:44:00.836729: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 06:44:00.836850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 06:44:00.837048: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 06:44:04.328710: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the follo

### Installing TF serving on colab environment

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  2943  100  2943    0     0   9118      0 --:--:-- --:--:-- --:--:--  9139
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,026 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://storage.googleapis.com/tensorflow-serving-apt stable/tensor

In [ ]:
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-2.8.0/t/tensorflow-model-server/tensorflow-model-server_2.8.0_all.deb'
!dpkg -i tensorflow-model-server_2.8.0_all.deb
!pip3 install tensorflow-serving-api==2.8.0

--2023-11-07 06:44:53--  http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-2.8.0/t/tensorflow-model-server/tensorflow-model-server_2.8.0_all.deb
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.12.27, 172.217.15.251, 172.217.164.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.12.27|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 340152790 (324M) [application/x-debian-package]
Saving to: ‘tensorflow-model-server_2.8.0_all.deb’

tensorflow-model-se 100%[===================>] 324.39M  52.0MB/s    in 7.7s    

2023-11-07 06:45:01 (42.4 MB/s) - ‘tensorflow-model-server_2.8.0_all.deb’ saved [340152790/340152790]

Selecting previously unselected package tensorflow-model-server.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack tensorflow-model-server_2.8.0_all.deb ...
Unpacking tensorflow-model-server (2.8.0) ...
Setting up tensorflow-model-serv

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
MODEL_DIR

'/tmp'

### Starting TF model server

In [ ]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=abs_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [ ]:
!tail server.log

2023-11-07 06:50:10.946639: E external/org_tensorflow/tensorflow/core/grappler/optimizers/meta_optimizer.cc:828] tfg_optimizer{} failed: NOT_FOUND: Op type not registered 'DisableCopyOnRead' in binary running on 0935925c8506. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
	when importing GraphDef to MLIR module in GrapplerHook
2023-11-07 06:50:11.791063: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 65798144 exceeds 10% of free system memory.
2023-11-07 06:50:12.257032: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 65798144 exceeds 10% of free system memory.
2023-11-07 06:50:12.641874: W external/org_tensorflow/tensorflow/core/framewor

In [ ]:
import requests
import json
import numpy as np

### Generating summary using TF serving
* We define the rest_api url at which the model is served and its format.
* We take the baseline article and pre-process it.
* The data is then convered into a format suitable to sent to the serving.
* The data is changed to json format and rest api request is sent.
* After the model generates summary, it is read and decoded into human readable format.

In [ ]:
tf_serving_url = "http://localhost:8501/v1/models/abs_model:predict"

article = 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we\'re neglected or forgotten," she said.\n"That may not be true but it is perhaps my perspective over the last few days.\n"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"\nMeanwhile, a flood alert remains in place across the Borders because of the constant rain.\nPeebles was badly hit by problems, sparking calls to introduce more defences in the area.\nScottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.\nThe Labour Party\'s deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.\nHe said it was important to get the flood protection plan right but backed calls to speed up the process.\n"I was quite taken aback by the amount of damage that has been done," he said.\n"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."\nHe said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.\nHave you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.'
print("Article: ")
print(article)
article = "summarize: " + article
tokenized = tokenizer([article], max_length=max_input_length, truncation=True, return_tensors='np')

input_data = {
    'input_ids': [tokenized['input_ids'][0].tolist()],
    'attention_mask': [tokenized['attention_mask'][0].tolist()]
}

#Define the input data for TensorFlow Serving
data = {
    'signature_name': 'serving_default',
    'instances': [input_data]
}

# Send the HTTP POST request
response = requests.post(tf_serving_url, data=json.dumps(data))

result = json.loads(response.content)
pred_tokens = result[0]['output_0']
generated_summary = tokenizer.decode(pred_tokens, skip_special_tokens=False)
print("Summary:")
print(generated_summary)

Article: 
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - bu

In [ ]:
metadata_url = "http://localhost:8501/v1/models/abs_model/metadata"

# Send a request to get the model metadata
response = requests.get(metadata_url)

# Parse the JSON response
metadata = response.json()

# Print the signature information
print(metadata)

{'model_spec': {'name': 'abs_model', 'signature_name': '', 'version': '1'}, 'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'input_ids': {'dtype': 'DT_INT32', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, {'size': '-1', 'name': ''}], 'unknown_rank': False}, 'name': 'serving_default_input_ids:0'}, 'attention_mask': {'dtype': 'DT_INT32', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, {'size': '-1', 'name': ''}], 'unknown_rank': False}, 'name': 'serving_default_attention_mask:0'}, 'decoder_attention_mask': {'dtype': 'DT_INT32', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, {'size': '-1', 'name': ''}], 'unknown_rank': False}, 'name': 'serving_default_decoder_attention_mask:0'}, 'decoder_input_ids': {'dtype': 'DT_INT32', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, {'size': '-1', 'name': ''}], 'unknown_rank': False}, 'name': 'serving_default_decoder_input_ids:0'}}, 'outputs': {'past_key_values_3_4': {'dtype': 'DT_FLOAT', 'tensor_sh

In [ ]:
!ps aux | grep tensorflow_model_server

root       27598  0.2  7.7 2306544 1029000 ?     Sl   06:50   0:05 tensorflow_model_server --rest_ap
root       37505  0.0  0.0   6616  2408 ?        S    07:31   0:00 grep tensorflow_model_server
